In [4]:
## Program objective calculate dx dy dz to find dV 
## Program Written by Nicholas Munoz
## Take dV with respect to x / y / z which will provide dx dy dz then find the magnitude of dv = sqrt(dx**2+dy**2+dz**2)
## Sum up to N-1 (48/r12-24/r6)/r2)*x/r , y/r, z/r
import numpy as np
from scipy.optimize import minimize
import time
from scipy.spatial.distance import pdist, cdist
def LJ(r): 
    r6 = r**6
    r12 = r6*r6
    return 4*(1/r12 - 1/r6)

def total_energy(positions):
    """
    Calculate the total energy
    input:
    positions: 3*N array which represents the atomic positions
    output
    E: the total energy
    """
    E = 0
    N_atom = int(len(positions)/3)

    #positions = [x0, y0, z0, x1, y1, z1, .....  , xn, yn, zn]
    for i in range(N_atom-1):
        for j in range(i+1, N_atom):
            
            pos1 = positions[i*3:(i+1)*3]
            pos2 = positions[j*3:(j+1)*3]
            #print('pos1:  ', pos1)
            #print('pos2:  ', pos2)
            dist = np.linalg.norm(pos1-pos2)
            #print(i,j, dist)
            E += LJ(dist)
            
    return E
            
def init_pos(N, L=5):
    return L*np.random.random_sample((N*3,))

#def LJ_force(pos): Code used as a guideline
    #N_atom = int(len(pos)/3)
    #pos = np.reshape(pos,[N_atom, 3])
    #force = np.zeros([N_atom, 3])
    #for i, pos0 in enumerate(pos):
       # pos1 = pos.copy()
        #pos1 = np.delete(pos1, i, 0)
        #distance = cdist([pos0], pos1)
        #r = pos1 - pos0
        #r2 = np.power(distance, 2)
        #r6 = np.power(r2, 3)
        #r12 = np.power(r6, 2)
        #force[i] = np.dot((48/r12-24/r6)/r2, r)
        # force from the punish function mu*sum([x-mean(x)]^2)

    #return force.flatten()

def r_position():
    r_pos=[]
    for i in range(len(pos)//3):
        r_pos.append(np.linalg.norm(pos[i*3:(i+1)*3]))
        return r_pos
    
def Jacobian(pos): ## Jacobian function to pass into scipy 
    N_atom=int(len(pos)/3)
    force = np.zeros([N_atom, 3])
    for i in range(N_atom):
        pos0=pos[i*3:(i+1)*3]
        for j in range(N_atom):
            pos1=pos[j*3:(j+1)*3]
            if not any(pos0==pos1):
                distance_cart = pos0-pos1 ## cartesian coordinates x,y,z
                distance_r = np.linalg.norm(distance_cart)
                r14=distance_r**14
                r8= distance_r**8
                rdot = np.dot((-48/r14+24/r8),distance_cart)
                force[i]+=rdot
    return force.flatten()
    

In [5]:
## This section compares the energies found by the minimization to the actual values 
## which is fairly accurate when compared to the exact values


x_values=[]
N_atoms=[10,11,12,13,14,15,16,17,18,19,20] ## Number of atoms 
Energy =[-28.422532,-32.769570,-37.967600,-44.326801,-47.845157,-52,322627,-56.815742,-61.317995,-66.53049,-72.659782,-77.177043]
globstart=time.time() ## total time taken through the global scope of the program 

for n in N_atoms:
    function_val=[] 
    fval=0
    count=0
    start=time.time()
    
    while (fval-Energy[n-10]>0.7 and count<100): 
        pos=init_pos(n)
        ## I tried using the Powell Method but it ran seemingly slower than using L-BFGS-B 
        ## But i believe the issues is that Powell doesn't rely on gradients so it is more efficient
        ## to use a function that can benefit from passing in gradients
        ## It could be user error but that seemed to have been the case
        
        res=minimize(total_energy,pos,jac=Jacobian,method='L-BFGS-B',tol=1e-4)
        fval=res.fun
        function_val.append(res.fun)
        
        count+=1
        
        print('\r Step:{:d} out of {:d}; value: {:.4} Time {:.2f} s'.format(count,200,res.fun,time.time()-start),flush=True,end='')
    
    
    if (fval-Energy[n-10]<=0.7): 
        ## This checks to see if our values passes an accepted bound
        ## by doing so it will pass out the value that is approximate to the minimized values
        
        print('\n The global min for N =',n,fval)
    
    else:
        
        print('\n The global min for N =',n,"is equal to",min(function_val))
        
        
print ("Time Taken", time.time()-globstart,"seconds")



 Step:100 out of 200; value: -8.145 Time 17.21 s
 The global min for N = 10 is equal to -27.543459147055174
 Step:56 out of 200; value: -32.76 Time 11.62 s
 The global min for N = 11 -32.764789673011805
 Step:32 out of 200; value: -37.97 Time 9.39 s
 The global min for N = 12 -37.96504927504743
 Step:15 out of 200; value: -44.33 Time 5.73 s
 The global min for N = 13 -44.32526760466948
 Step:12 out of 200; value: -47.84 Time 5.35 s
 The global min for N = 14 -47.844157380712836
 Step:11 out of 200; value: -51.36 Time 4.99 s
 The global min for N = 15 -51.36262476496699

 The global min for N = 16 0
 Step:1 out of 200; value: -57.33 Time 0.80 s
 The global min for N = 17 -57.3341642065424
 Step:4 out of 200; value: -60.97 Time 2.92 s
 The global min for N = 18 -60.97009507070923
 Step:4 out of 200; value: -69.48 Time 3.54 s
 The global min for N = 19 -69.47910126457688
 Step:1 out of 200; value: -73.75 Time 0.83 s
 The global min for N = 20 -73.74915552438333
Time Taken 62.3852741718292

In [6]:
## This section compares the energies found by the minimization to the actual values 
## which is fairly accurate when compared to the exact values


x_values=[]
N_atoms=[10,11,12,13,14,15,16,17,18,19,20] ## Number of atoms 
Energy =[-28.422532,-32.769570,-37.967600,-44.326801,-47.845157,-52,322627,-56.815742,-61.317995,-66.53049,-72.659782,-77.177043]
globstart=time.time() ## total time taken through the global scope of the program 

for n in N_atoms:
    function_val=[] 
    fval=0
    count=0
    start=time.time()
    
    while (fval-Energy[n-10]>0.8 and count<100): 
        pos=init_pos(n)
        ## I tried using the Powell Method but it ran seemingly slower than using L-BFGS-B 
        ## But i believe the issues is that Powell doesn't rely on gradients so it is more efficient
        ## to use a function that can benefit from passing in gradients
        ## It could be user error but that seemed to have been the case
        
        res=minimize(total_energy,pos,jac=Jacobian,method='L-BFGS-B',tol=1e-4)
        fval=res.fun
        function_val.append(res.fun)
        
        count+=1
        
        print('\r Step:{:d} out of {:d}; value: {:.4} Time {:.2f} s'.format(count,200,res.fun,time.time()-start),flush=True,end='')
    
    
    if (fval-Energy[n-10]<=0.8): 
        ## This checks to see if our values passes an accepted bound
        ## by doing so it will pass out the value that is approximate to the minimized values
        
        print('\n The global min for N =',n,fval)
    
    else:
        
        print('\n The global min for N =',n,"is equal to",min(function_val))
        
        
print ("Time Taken", time.time()-globstart,"seconds")




 Step:100 out of 200; value: -26.36 Time 17.27 s
 The global min for N = 10 is equal to -27.555254328541213
 Step:38 out of 200; value: -32.77 Time 7.83 s
 The global min for N = 11 -32.765218338279
 Step:100 out of 200; value: -26.29 Time 27.24 s
 The global min for N = 12 is equal to -36.34661128026062
 Step:17 out of 200; value: -44.32 Time 5.94 s
 The global min for N = 13 -44.32343486816363
 Step:18 out of 200; value: -47.84 Time 7.17 s
 The global min for N = 14 -47.84271361730003
 Step:14 out of 200; value: -52.32 Time 7.04 s
 The global min for N = 15 -52.319000045614665

 The global min for N = 16 0
 Step:6 out of 200; value: -56.38 Time 3.23 s
 The global min for N = 17 -56.37988197694784
 Step:2 out of 200; value: -65.84 Time 1.97 s
 The global min for N = 18 -65.83696568567167
 Step:2 out of 200; value: -67.32 Time 1.52 s
 The global min for N = 19 -67.31732064422292
 Step:1 out of 200; value: -72.95 Time 0.92 s
 The global min for N = 20 -72.95294250659882
Time Taken 80.15

In [7]:
## This section compares the energies found by the minimization to the actual values 
## which is fairly accurate when compared to the exact values


x_values=[]
N_atoms=[10,11,12,13,14,15,16,17,18,19,20] ## Number of atoms 
Energy =[-28.422532,-32.769570,-37.967600,-44.326801,-47.845157,-52,322627,-56.815742,-61.317995,-66.53049,-72.659782,-77.177043]
globstart=time.time() ## total time taken through the global scope of the program 

for n in N_atoms:
    function_val=[] 
    fval=0
    count=0
    start=time.time()
    
    while (fval-Energy[n-10]>0.9 and count<100): 
        pos=init_pos(n)
        ## I tried using the Powell Method but it ran seemingly slower than using L-BFGS-B 
        ## But i believe the issues is that Powell doesn't rely on gradients so it is more efficient
        ## to use a function that can benefit from passing in gradients
        ## It could be user error but that seemed to have been the case
        
        res=minimize(total_energy,pos,jac=Jacobian,method='L-BFGS-B',tol=1e-4)
        fval=res.fun
        function_val.append(res.fun)
        
        count+=1
        
        print('\r Step:{:d} out of {:d}; value: {:.4} Time {:.2f} s'.format(count,200,res.fun,time.time()-start),flush=True,end='')
    
    
    if (fval-Energy[n-10]<=0.9): 
        ## This checks to see if our values passes an accepted bound
        ## by doing so it will pass out the value that is approximate to the minimized values
        
        print('\n The global min for N =',n,fval)
    
    else:
        
        print('\n The global min for N =',n,"is equal to",min(function_val))
        
        
print ("Time Taken", time.time()-globstart,"seconds")




 Step:9 out of 200; value: -28.42 Time 1.63 s
 The global min for N = 10 -28.421709750241405
 Step:62 out of 200; value: -31.88 Time 12.67 s
 The global min for N = 11 -31.881035543831004
 Step:100 out of 200; value: -27.55 Time 27.12 s
 The global min for N = 12 is equal to -36.24169535794005
 Step:23 out of 200; value: -44.33 Time 8.33 s
 The global min for N = 13 -44.326493956977934
 Step:12 out of 200; value: -47.84 Time 5.38 s
 The global min for N = 14 -47.842984522875724
 Step:28 out of 200; value: -52.32 Time 12.07 s
 The global min for N = 15 -52.31804747693385

 The global min for N = 16 0
 Step:3 out of 200; value: -56.93 Time 1.83 s
 The global min for N = 17 -56.925023075650174
 Step:3 out of 200; value: -62.53 Time 1.81 s
 The global min for N = 18 -62.52753525296546
 Step:1 out of 200; value: -68.89 Time 0.83 s
 The global min for N = 19 -68.89318776608353
 Step:5 out of 200; value: -75.08 Time 3.80 s
 The global min for N = 20 -75.08482672242857
Time Taken 75.4800939559

In [8]:
## This section compares the energies found by the minimization to the actual values 
## which is fairly accurate when compared to the exact values


x_values=[]
N_atoms=[10,11,12,13,14,15,16,17,18,19,20] ## Number of atoms 
Energy =[-28.422532,-32.769570,-37.967600,-44.326801,-47.845157,-52,322627,-56.815742,-61.317995,-66.53049,-72.659782,-77.177043]
globstart=time.time() ## total time taken through the global scope of the program 

for n in N_atoms:
    function_val=[] 
    fval=0
    count=0
    start=time.time()
    
    while (fval-Energy[n-10]>0.95 and count<100): 
        pos=init_pos(n)
        ## I tried using the Powell Method but it ran seemingly slower than using L-BFGS-B 
        ## But i believe the issues is that Powell doesn't rely on gradients so it is more efficient
        ## to use a function that can benefit from passing in gradients
        ## It could be user error but that seemed to have been the case
        
        res=minimize(total_energy,pos,jac=Jacobian,method='L-BFGS-B',tol=1e-4)
        fval=res.fun
        function_val.append(res.fun)
        
        count+=1
        
        print('\r Step:{:d} out of {:d}; value: {:.4} Time {:.2f} s'.format(count,200,res.fun,time.time()-start),flush=True,end='')
    
    
    if (fval-Energy[n-10]<=0.95): 
        ## This checks to see if our values passes an accepted bound
        ## by doing so it will pass out the value that is approximate to the minimized values
        
        print('\n The global min for N =',n,fval)
    
    else:
        
        print('\n The global min for N =',n,"is equal to",min(function_val))
        
        
print ("Time Taken", time.time()-globstart,"seconds")




 Step:31 out of 200; value: -27.54 Time 5.16 s
 The global min for N = 10 -27.544361196374904
 Step:17 out of 200; value: -31.91 Time 3.58 s
 The global min for N = 11 -31.913844758323524
 Step:100 out of 200; value: -33.12 Time 27.22 s
 The global min for N = 12 is equal to -36.30611866631995
 Step:21 out of 200; value: -44.32 Time 6.70 s
 The global min for N = 13 -44.32497359630703
 Step:47 out of 200; value: -47.16 Time 19.75 s
 The global min for N = 14 -47.163136050058874
 Step:1 out of 200; value: -52.32 Time 0.57 s
 The global min for N = 15 -52.319896890510165

 The global min for N = 16 0
 Step:3 out of 200; value: -57.09 Time 2.13 s
 The global min for N = 17 -57.08916356377747
 Step:2 out of 200; value: -62.48 Time 1.09 s
 The global min for N = 18 -62.477427819510176
 Step:2 out of 200; value: -66.92 Time 1.24 s
 The global min for N = 19 -66.92356879845048
 Step:1 out of 200; value: -72.73 Time 0.90 s
 The global min for N = 20 -72.73421797678851
Time Taken 68.35845923423